# This is an implemetation of ADALINE

The code down below is part of the *lib.py* file, which will contain the:
* random weights generation
* the prediction function *threshold function*
* the weighted sum function
* and the training function

In [83]:
import numpy as np

In [84]:
def random_weights(X, random_state: int):
    rand = np.random.RandomState(random_state)
    w = rand.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
    return w

In [85]:
def predict(X, w):
    return np.where(net_input(X, w) >= 0.0, 1, -1)

In [86]:
def net_input(X, w):
    return np.dot(X, w[1:]) + w[0]

In [87]:
def fit(X, y, eta=0.001, n_iter=1):
    '''loop over exemplars and update weights'''
    mse_iteration = []
    w = random_weights(X, random_state=1)
    for pair in range(n_iter):
        output = net_input(X, w)
        gradient = 2*(output - y)
        w[1:] += eta*(X.T @-gradient)
        w[0] += eta*-gradient.sum()
        mse = (((y - output)**2).sum())/len(y)
        mse_iteration.append(mse)
    return w, mse_iteration

In [88]:
import numpy as np
import pandas as pd
import altair as alt

In [89]:
def species_generator(mu1, sigma1, mu2, sigma2, n_samples, target, seed):
    # mu1 is the mean weight
    # mu2 is the  mean wingspan
    rand = np.random.RandomState(seed)
    f1 = rand.normal(mu1, sigma1, n_samples)
    f2 = rand.normal(mu2, sigma2, n_samples)
    X = np.array([f1, f2])
    X = X.transpose()
    y = np.full((n_samples), target)
    return X, y

In [90]:
albatross_weight_mean = 9000 # in grams
albatross_weight_variance =  800 # in grams
albatross_wingspan_mean = 300 # in cm
albatross_wingspan_variance = 20 # in cm 
n_samples = 100
target = 1
seed = 100

# aX: feature matrix (weight, wingspan)
# ay: target value (1)
aX, ay = species_generator(albatross_weight_mean, albatross_weight_variance,
                           albatross_wingspan_mean, albatross_wingspan_variance,
                           n_samples,target,seed )


albatross_dic = {'weight-(gm)': aX[:,0],
                 'wingspan-(cm)': aX[:,1], 
                 'species': ay,
                 'url': "https://raw.githubusercontent.com/pabloinsente/nn-mod-cog/master/notebooks/images/albatross.png"}


# put values in a relational table (pandas dataframe)
albatross_df = pd.DataFrame(albatross_dic)

In [91]:
owl_weight_mean = 1000 # in grams
owl_weight_variance =  200 # in grams
owl_wingspan_mean = 100 # in cm
owl_wingspan_variance = 15 # in cm
n_samples = 100
target = -1
seed = 100

# oX: feature matrix (weight, wingspan)
# oy: target value (1)
oX, oy = species_generator(owl_weight_mean, owl_weight_variance,
                           owl_wingspan_mean, owl_wingspan_variance,
                           n_samples,target,seed )

owl_dic = {'weight-(gm)': oX[:,0],
             'wingspan-(cm)': oX[:,1], 
             'species': oy,
             'url': "https://raw.githubusercontent.com/pabloinsente/nn-mod-cog/master/notebooks/images/owl.png"}

# put values in a relational table (pandas dataframe)
owl_df = pd.DataFrame(owl_dic)

In [92]:
# df = albatross_df.append(owl_df, ignore_index=True)
df = pd.concat([albatross_df, owl_df], ignore_index=True)

alt.Chart(df).mark_image(
    width=20,
    height=20
).encode(
    x="weight-(gm)",
    y="wingspan-(cm)",
    url="url"
).properties(
    title='Chart 1'
)

alt.Chart(...)

In [93]:
df_shuffle = df.sample(frac=1, random_state=1).reset_index(drop=True)
X = df_shuffle[['weight-(gm)','wingspan-(cm)']].to_numpy()
y = df_shuffle['species'].to_numpy()

In [94]:
w, mse = fit(X, y, eta=1e-10, n_iter=12)
print(w, mse)

[ 0.01624407  0.00024607 -0.00507522] [1657.7059195090953, 645.6347772640765, 251.58682683094383, 98.16500374231856, 38.43049519101968, 15.172960286729813, 6.1176641091291435, 2.591981953502288, 1.2192442640539063, 0.6847513186984904, 0.47662733844914024, 0.39557448463381123]


In [95]:
y_pred = predict(X, w)
num_correct_predictions = (y_pred == y).sum()
accuracy = (num_correct_predictions / y.shape[0]) * 100
print('ADALINE accuracy: %.2f%%' % accuracy)

ADALINE accuracy: 99.50%


In [96]:
mse_df = pd.DataFrame({'mse':mse, 'time-step': np.arange(0, len(mse))})
base = alt.Chart(mse_df).encode(x="time-step:O")
base.mark_line().encode(
    y="mse"
).properties(
    width=400,
    title='Chart 2'
)


alt.Chart(...)

In [97]:
# condor_weight_mean = 12000 # in grams
# condor_weight_variance = 1000 # in grams
# condor_wingspan_mean = 290 # in cm
# condor_wingspan_variance = 15 # in cm 
# n_samples = 100
# target = -1
# seed = 100

# # cX: feature matrix (weight, wingspan)
# # cy: target value (1)
# cX, cy = species_generator(condor_weight_mean, condor_weight_variance,
#                            condor_wingspan_mean, condor_wingspan_variance,
#                            n_samples,target,seed )

# condor_dic = {'weight-(gm)': cX[:,0],
#              'wingspan-(cm)': cX[:,1], 
#              'species': cy,
#              'url': "https://raw.githubusercontent.com/pabloinsente/nn-mod-cog/master/notebooks/images/condor.png"}

# # put values in a relational table (pandas dataframe)
# condor_df = pd.DataFrame(condor_dic)

# df2 = pd.concat([albatross_df, condor_df], ignore_index=True)

In [98]:
# alt.Chart(df2).mark_image(
#     width=20,
#     height=20
# ).encode(
#     alt.X("weight-(gm)", scale=alt.Scale(domain=(6000, 16000))),
#     alt.Y("wingspan-(cm)", scale=alt.Scale(domain=(220, 360))),
#     url="url"
# ).properties(
# title="Chart 3"
# )

In [99]:
# df_shuffle2 = df2.sample(frac=1, random_state=1).reset_index(drop=True)
# X = df_shuffle2[['weight-(gm)','wingspan-(cm)']].to_numpy()
# y = df_shuffle2['species'].to_numpy()

In [100]:
# w, errors = fit(X, y, eta=0.01, n_iter=1000)

In [101]:
# y_pred = predict(X, w)
# num_correct_predictions = (y_pred == y).sum()
# accuracy = (num_correct_predictions / y.shape[0]) * 100
# print('Perceptron accuracy: %.2f%%' % accuracy)